In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!pip install weaviate-client langchain tiktoken pypdf rapidocr-onnxruntime


In [ ]:
# For beta versions: `pip install --pre -U "weaviate-client==4.*"`
! pip install  weaviate-client

In [ ]:
import weaviate
from weaviate.classes.init import Auth

In [ ]:
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url,
    auth_credentials=Auth.api_key(weaviate_api_key),
)
print(client.is_ready())

In [ ]:
!pip install sentence-transformers

In [ ]:
!pip install langchain_community

In [ ]:
# fixing unicode error in colab
import locale
locale.getpreferredencoding=lambda: "UTF-8"

In [ ]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
embedding_model_name="sentence-transformers/all-mpnet-base-v2"
embeddedings=HuggingFaceBgeEmbeddings(model_name=embedding_model_name)

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader("/content/rag.pdf",extract_images=True)
pages=loader.load()

In [ ]:
pages

In [ ]:
# Split text into chunks
from langchain.text_splitter import CharacterTextSplitter
text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=20)
docs=text_splitter.split_documents(pages)

In [ ]:
docs

In [ ]:
 #install package
 %pip install -Uqq langchain-weaviate

In [ ]:
import weaviate
from langchain_weaviate.vectorstores import WeaviateVectorStore

In [ ]:
db = WeaviateVectorStore.from_documents(docs, embeddedings,client=client)

In [ ]:
query = "what is rag?"
docs = db.similarity_search(query)

In [ ]:
print(db.similarity_search("what is rag?", k=3)[1].page_content)

In [ ]:
print(db.similarity_search("what is rag?", k=3)[2].page_content)

In [ ]:
print(
    db.similarity_search(
        "what is attention?", k=3)
    )


In [ ]:
from langchain.prompts import ChatPromptTemplate

template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [ ]:
prompt=ChatPromptTemplate.from_template(template)

In [ ]:
prompt

In [ ]:
! pip install -U langchain-huggingface

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint

In [ ]:
model = HuggingFaceEndpoint(
    timeout=400,
    repo_id="mistralai/Mistral-Nemo-Instruct-2407",
    temperature=1,max_length=180,token=huggingfacehub_api_token

)

In [ ]:
model

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [ ]:
output_parser=StrOutputParser()

In [ ]:
retriever=db.as_retriever()

In [ ]:
rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

In [ ]:
print(rag_chain.invoke("what is rag system?"))